In [14]:
import torch
torch.__version__

'2.0.1+cu117'

In [15]:
import numpy as np
import pandas as pd
from glob import glob
import json

In [16]:
train_list = glob('/home/work/dataADD/ktelspeech/ksponspeech/train/*/*/*.pcm')
train_text = glob('/home/work/dataADD/ktelspeech/ksponspeech/train/*/*/*.txt')
train_list.sort()
train_text.sort()

In [17]:
len(train_list)

622545

In [18]:
train_list[:10]

['/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000001.pcm',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000002.pcm',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000003.pcm',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000004.pcm',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000005.pcm',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000006.pcm',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000007.pcm',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000008.pcm',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000009.pcm',
 '/home/work/dataADD/ktelspeech/kspon

In [19]:
train_text[:10]

['/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000001.txt',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000002.txt',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000003.txt',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000004.txt',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000005.txt',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000006.txt',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000007.txt',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000008.txt',
 '/home/work/dataADD/ktelspeech/ksponspeech/train/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000009.txt',
 '/home/work/dataADD/ktelspeech/kspon

In [20]:
train_labels = pd.read_csv('/home/work/dataADD/ktelspeech/Training/D60/train_label.csv',encoding='utf-8')

In [21]:
len(train_labels)

593643

In [22]:
import re
def bracket_filter(sentence):
    new_sentence = str()
    flag = False
    
    for ch in sentence:
        if ch == '(' and flag == False: 
            flag = True
            continue
        if ch == '(' and flag == True:
            flag = False
            continue
        if ch != ')' and flag == False:
            new_sentence += ch
    return new_sentence

In [23]:
def special_filter(sentence):
    SENTENCE_MARK = ['?', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', '.', ',']
    
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            # o/, n/ 등 처리
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
                continue 

        if ch == '#': 
            new_sentence += '샾'

        elif ch not in EXCEPT: 
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence

In [24]:
def sentence_filter(raw_sentence):
    return special_filter(bracket_filter(raw_sentence))

In [25]:
from tqdm import tqdm
trans_list = list()
for i in tqdm(range(len(train_list))):
    try :
        with open(train_text[i], "r",encoding='euc-kr') as file:
            lines = file.readlines()
            for line in lines :
#                print(sentence_filter(line))
                trans_list.append(sentence_filter(line))
    except :
        with open(train_text[i], "r",encoding='cp949') as file:
            lines = file.readlines()
            for line in lines :
#                print(sentence_filter(line))
                trans_list.append(sentence_filter(line))

100% 622545/622545 [03:02<00:00, 3403.61it/s]


In [27]:
train_labels.head()

,filename,text
0,S00000001_0001,안녕하세요. 쉐어링 스터디 상담원 @이주빈입니다.
1,S00000001_0002,예 안녕하세요 제가 다른 게 아니라 나이가 좀 있는 지라 강좌를 결재 방법이 조금 ...
2,S00000001_0003,그거 (SHARING)/(쉐어링) (CASH)/(캐시)충전이었나
3,S00000001_0004,그게 처음에 (MY PAGE)/(마이 페이지)를 들어가서 (CASH)/(캐시)를 버...
4,S00000001_0005,(CASH)/(캐시) 할인권 (MENU)/(메뉴)에서 (SHARING CASH)/(...


In [28]:
import pandas as pd

dic = {"filename":train_list,"text":trans_list}
new_train_labels = pd.DataFrame(dic)

In [30]:
len(new_train_labels)

622545

In [29]:
new_train_labels.head()

,filename,text
0,/home/work/dataADD/ktelspeech/ksponspeech/trai...,아 몬 소리야 그건 또
1,/home/work/dataADD/ktelspeech/ksponspeech/trai...,나는 악습은 원래 없어진다 없어져야 된다고 생각하긴 했는데 근데 그 약간 필요악으로...
2,/home/work/dataADD/ktelspeech/ksponspeech/trai...,그래서 지호랑 계단 올라와서 막 위에 운동하는 기구 있대요 그서 그걸로 운동 할려구요
3,/home/work/dataADD/ktelspeech/ksponspeech/trai...,뭐 정신과 병원도 그 약 타서 먹어보고 그 한동안 연락이 안 된 적이 있었단 말이야...
4,/home/work/dataADD/ktelspeech/ksponspeech/trai...,그게 영 점 일 프로 가정의 아이들과 가정의 모습이야?


In [31]:
new_train_labels.to_csv('/home/work/dataADD/ktelspeech/ksponspeech/train/train_label.csv',index = False,encoding='utf-8-sig')